<a href="https://colab.research.google.com/github/sumiyaanika/roadway-impairment/blob/main/gau_Lap_cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Display an image
def display_image(img, title="Image"):
    plt.figure(figsize=(6, 6))
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Display multiple images in a row for pyramids
def display_pyramid(pyramid, title="Pyramid"):
    levels = len(pyramid)
    plt.figure(figsize=(15, 5))
    for i, img in enumerate(pyramid):
        plt.subplot(1, levels, i+1)
        plt.imshow(img, cmap='gray')
        plt.title(f"{title} Level {i}")
        plt.axis('off')
    plt.show()

# Load an image
from google.colab import files
uploaded = files.upload()

# Set the image path and load the image
image_path = next(iter(uploaded))
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Display the original image
display_image(image, "Original Image")

# Set the levels based on image size
levels = 5 if min(image.shape) > 512 else 4

# Gaussian Pyramid
gaussian_pyramid = [image]
for i in range(1, levels):
    downsampled = cv2.pyrDown(gaussian_pyramid[-1])
    gaussian_pyramid.append(downsampled)

# Display Gaussian Pyramid
display_pyramid(gaussian_pyramid, title="Gaussian Pyramid")

# Laplacian Pyramid
laplacian_pyramid = []
for i in range(levels - 1):
    # Calculate the correct size for upsampling
    size = (gaussian_pyramid[i].shape[1], gaussian_pyramid[i].shape[0])  # Width, Height
    upsampled = cv2.pyrUp(gaussian_pyramid[i + 1], dstsize=size)
    laplacian = cv2.subtract(gaussian_pyramid[i], upsampled)
    laplacian_pyramid.append(laplacian)

laplacian_pyramid.append(gaussian_pyramid[-1])  # The smallest Gaussian level is added as the last Laplacian level

# Display Laplacian Pyramid
display_pyramid(laplacian_pyramid, title="Laplacian Pyramid")


In [ ]:
#include <opencv2/opencv.hpp>
#include <iostream>
#include <vector>

using namespace cv;
using namespace std;

// Display multiple images in a window for pyramids
void displayPyramid(const vector<Mat>& pyramid, const string& title) {
    int levels = pyramid.size();
    int windowWidth = 800; // Adjust window width
    Mat combined;

    // Combine all levels horizontally for display
    for (int i = 0; i < levels; ++i) {
        Mat resized;
        double scale = static_cast<double>(windowWidth) / pyramid[0].cols;
        resize(pyramid[i], resized, Size(), scale, scale);
        if (i == 0) {
            combined = resized.clone();
        } else {
            hconcat(combined, resized, combined);
        }
    }

    imshow(title, combined);
    waitKey(0);
}

int main() {
    // Load the image in grayscale
    string imagePath;
    cout << "Enter the path to the image: ";
    cin >> imagePath;

    Mat image = imread(imagePath, IMREAD_GRAYSCALE);
    if (image.empty()) {
        cerr << "Error: Unable to load image!" << endl;
        return -1;
    }

    // Display the original image
    imshow("Original Image", image);
    waitKey(0);

    // Set levels based on image size
    int levels = (min(image.rows, image.cols) > 512) ? 5 : 4;

    // Gaussian Pyramid
    vector<Mat> gaussianPyramid;
    gaussianPyramid.push_back(image);
    for (int i = 1; i < levels; ++i) {
        Mat downsampled;
        pyrDown(gaussianPyramid[i - 1], downsampled);
        gaussianPyramid.push_back(downsampled);
    }

    // Display Gaussian Pyramid
    displayPyramid(gaussianPyramid, "Gaussian Pyramid");

    // Laplacian Pyramid
    vector<Mat> laplacianPyramid;
    for (int i = 0; i < levels - 1; ++i) {
        Mat upsampled, laplacian;
        pyrUp(gaussianPyramid[i + 1], upsampled, gaussianPyramid[i].size());
        subtract(gaussianPyramid[i], upsampled, laplacian);
        laplacianPyramid.push_back(laplacian);
    }
    laplacianPyramid.push_back(gaussianPyramid.back()); // Add smallest Gaussian level

    // Display Laplacian Pyramid
    displayPyramid(laplacianPyramid, "Laplacian Pyramid");

    return 0;
}


SyntaxError: invalid syntax (<ipython-input-2-af56337820d2>, line 5)